<a href="https://colab.research.google.com/github/2303A52450/statml/blob/main/assignment_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load the Dataset
# Replace 'breast_cancer.csv' with the correct dataset file path
data = pd.read_csv('breast_cancer_survival.csv')

# 2. Identify Features and Target Variable
# Check the dataset structure
print(data.info())
print(data.head())

# Assuming 'diagnosis' is the target column (adjust based on actual column names)
target = 'diagnosis'
features = data.drop(columns=[target]).columns

X = data[features]
y = data[target]

# Convert target to numerical if it contains categorical labels (e.g., malignant/benign)
if y.dtypes == 'object':
    y = y.map({'M': 1, 'B': 0})  # Update based on the dataset's encoding

# 3. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

# Train models and calculate accuracy
accuracies = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies[name] = acc
    print(f"{name} Accuracy: {acc:.2f}")

# 4. Bootstrapping
bootstrap_results = {}
n_iterations = 1000

for name, model in models.items():
    bootstrap_acc = []
    for _ in range(n_iterations):
        # Resample training data with replacement
        indices = np.random.choice(len(X_train), len(X_train), replace=True)
        X_resampled = X_train.iloc[indices]
        y_resampled = y_train.iloc[indices]
        # Train and evaluate on test data
        model.fit(X_resampled, y_resampled)
        y_pred = model.predict(X_test)
        bootstrap_acc.append(accuracy_score(y_test, y_pred))
    bootstrap_results[name] = bootstrap_acc

# 5. Plot Bootstrapping Results
plt.figure(figsize=(12, 6))
for name, accs in bootstrap_results.items():
    sns.kdeplot(accs, label=name, shade=True)
plt.title('Bootstrapping Accuracy Distributions')
plt.xlabel('Accuracy')
plt.ylabel('Density')
plt.legend()
plt.show()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 334 non-null    int64  
 1   Gender              334 non-null    object 
 2   Protein1            334 non-null    float64
 3   Protein2            334 non-null    float64
 4   Protein3            334 non-null    float64
 5   Protein4            334 non-null    float64
 6   Tumour_Stage        334 non-null    object 
 7   Histology           334 non-null    object 
 8   ER status           334 non-null    object 
 9   PR status           334 non-null    object 
 10  HER2 status         334 non-null    object 
 11  Surgery_type        334 non-null    object 
 12  Date_of_Surgery     334 non-null    object 
 13  Date_of_Last_Visit  317 non-null    object 
 14  Patient_Status      321 non-null    object 
dtypes: float64(4), int64(1), object(10)
memory usage: 39.3+ K

KeyError: "['diagnosis'] not found in axis"